## Minimalist Example for HCB Domain Decomposition Method (fixed interface)

This prototype is meant to illustrate the steps of Hurty / Craig - Bampton domain decomposition procedure. Consider a system with the following mass & stiffness matrices and load vector:  

\begin{equation*}
    \mathbf{M} 
    =
    \begin{bmatrix} 
        1 & 0 & 0 & 0 & 0 & 0 \\
        0 & 2 & 0 & 0 & 0 & 0 \\
        0 & 0 & 2 & 0 & 0 & 0 \\
        0 & 0 & 0 & 2 & 0 & 0 \\
        0 & 0 & 0 & 0 & 2 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1
    \end{bmatrix} 
    \phantom{xx} 
    \text{;} 
    \phantom{xx}
    \mathbf{K} 
    =
    \begin{bmatrix} 
        1 & -1 & 0 & 0 & 0 & 0 \\
        -1 & 2 & -1 & 0 & 0 & 0 \\
        0 & -1 & 2 & -1 & 0 & 0 \\
        0 & 0 & -1 & 2 & -1 & 0 \\
        0 & 0 & 0 & -1 & 2 & -1 \\
        0 & 0 & 0 & 0 & -1 & 1
    \end{bmatrix} 
    \phantom{xx} 
    \text{;} 
    \phantom{xx}
    \mathbf{F}
    =
    \begin{bmatrix}
    1 \\ 1 \\ 1 \\ 1 \\ 1 \\ 1
    \end{bmatrix}
\end{equation*}

The system is decomposed into two parts; first part contains nodes $1$, $2$, and $3$ while the second part contains nodes $3$, $4$, $5$, and $6$. In other words, node $3$ is the boundary node. In addition, a dirichlet boundary condition is applied to node $6$ i.e. $u_{6}=0$

### Load

In [10]:
import numpy as np 
from scipy.linalg import sqrtm 

omega = 2.0

### Substructure 1 

The following computations are performed on computer 1 and are completely independent of substructure 2. Therefore the computation for substructure 2 can be performed in parallel. 

In [11]:
M_1 = np.array([[ 1, 0, 0],
                [ 0, 2, 0],
                [ 0, 0, 1]]) 

K_1 = np.array([[ 1,-1, 0],
                [-1, 2,-1],
                [ 0,-1, 1]])

f_1 = np.array([[1],
                [1],
                [1]])

# compute internal mode from eigenproblem with boundary node (node 3) fixed 

M_II_1 = M_1[0:2,0:2]
K_II_1 = K_1[0:2,0:2]

_, Phi_N_1 = np.linalg.eig( np.linalg.solve( M_II_1, K_II_1 ) )

# use mass - normalised eigenmodes 

Phi_N_1 = np.linalg.solve( sqrtm( (np.transpose(Phi_N_1) @ M_II_1 @ Phi_N_1)  ), Phi_N_1 ) 

# compute boundary mode

K_IB_1 = K_1[0:2,2]

Phi_C_1 = - np.reshape( np.linalg.solve( K_II_1, K_IB_1 ), (2,1) )

# compute transformation matrix 

I = np.eye( Phi_C_1.shape[1] )
O = np.zeros( (I.shape[0], Phi_N_1.shape[1]) )

G_1 = np.vstack(( np.hstack( (Phi_N_1, Phi_C_1) ),
                  np.hstack( (      O,       I) )))

# compute modal system of equations

K_1 = 1.2 * K_1 

D_1 = np.transpose(G_1) @ ( K_1 - omega * omega * M_1 ) @ G_1

# compute Schur complement matrix 

D_NN_1 = D_1[0:2,0:2] 
D_NC_1 = D_1[0:2,2] 
D_CN_1 = D_1[2,0:2] 
D_CC_1 = D_1[2,2]

S_1 =  D_CC_1 - D_CN_1 @ np.linalg.solve( D_NN_1, D_NC_1 ) 

# compute load vector 

F_1 = np.transpose(G_1) @ f_1

F_N_1 = F_1[0:2,0] 
F_C_1 = F_1[2,0] 

P_1 = F_C_1 - D_CN_1 @ np.linalg.solve( D_NN_1, F_N_1 ) 

### Substructure 2

The following computations are performed on computer 2 and are completely independent of substructure 1. Therefore the computation for substructure 1 can be performed in parallel. 

In [12]:
M_2 = np.array([[ 2, 0, 0, 0],
                [ 0, 2, 0, 0],
                [ 0, 0, 1, 0],
                [ 0, 0, 0, 1]]) 

K_2 = np.array([[ 2,-1, 0,-1],
                [-1, 2, 0, 0],
                [ 0, 0, 1, 0],
                [-1, 0, 0, 1]])

f_2 = np.array([[1],
                [1],
                [0],
                [0]])

# compute internal mode from eigenproblem with boundary node and clamped node (node 3 and 6) fixed 

M_II_2 = M_2[0:3,0:3]
K_II_2 = K_2[0:3,0:3]


_, Phi_N_2 = np.linalg.eig( np.linalg.solve( M_II_2, K_II_2 ) )

# use mass - normalised eigenmodes 

Phi_N_2 = np.linalg.solve( sqrtm( (np.transpose(Phi_N_2) @ M_II_2 @ Phi_N_2) ), Phi_N_2 ) 

# compute boundary mode

K_IB_2 = K_2[0:3,3]

Phi_C_2 = - np.reshape( np.linalg.solve( K_II_2, K_IB_2 ), (3,1) )

# # compute transformation matrix 

I = np.eye( Phi_C_2.shape[1] )
O = np.zeros( (I.shape[0], Phi_N_2.shape[1]) )

G_2 = np.vstack(( np.hstack( (Phi_N_2, Phi_C_2) ),
                  np.hstack( (      O,       I) )))

# # compute modal system of equations 

K_2 = np.array([[ 2.4,-1.2, 0.0,-1.2],
                [-1.2, 2.4, 0.0, 0.0],
                [ 0.0, 0.0, 1.0, 0.0],
                [-1.2, 0.0, 0.0, 1.2]])

D_2 = np.transpose(G_2) @ ( K_2 - omega * omega * M_2 ) @ G_2

# # compute Schur complement matrix 

D_NN_2 = D_2[0:3,0:3] 
D_NC_2 = D_2[0:3,3] 
D_CN_2 = D_2[3,0:3] 
D_CC_2 = D_2[3,3]

S_2 =  D_CC_2 - D_CN_2 @ np.linalg.solve( D_NN_2, D_NC_2 ) 

# # compute load vector 

F_2 = np.transpose(G_2) @ f_2

F_N_2 = F_2[0:3,0] 
F_C_2 = F_2[3,0] 

P_2 = F_C_2 - D_CN_2 @ np.linalg.solve( D_NN_2, F_N_2 ) 

### Solve for Boundary Freedoms

After Schur complement matrices and modified load vectors are computed, either computer 2 send the matrix and vector to computer 1 or vice versa to solve for boundary freedoms.  

In [19]:
u_C = (P_1+P_2) / (S_1+S_2)  

### Solve for internal mode 1

If boundary freedoms are computed in computer 2, the results need to be sent to computer 1 first. The following computation is independent of substructure 2. Therefore, computation in substructure 2 can be performed in parallel. 

In [14]:
u_N_1 = np.linalg.solve( D_NN_1, (F_N_1 - u_C * D_NC_1) )
u_1 = np.reshape( Phi_N_1 @ u_N_1, (2,1) ) + np.reshape( u_C * Phi_C_1, (2,1) )

### Solve for internal mode 2

If boundary freedoms are computed in computer 1, the results need to be sent to computer 2. The following computation is independent of substructure 1. Therefore, computation in substructure 1 can be performed in parallel. 

In [15]:
u_N_2 = np.linalg.solve( D_NN_2, (F_N_2 - u_C * D_NC_2) )
u_2 = np.reshape( Phi_N_2 @ u_N_2, (3,1) ) + np.reshape( u_C * Phi_C_2, (3,1) )

### Gather Solution

In the final step solutions from all computers are gathered to form the complete solution. 

In [16]:
u_HCB = np.vstack((np.vstack ((u_1,u_C)),u_2))
print(u_HCB)

[[-0.32278686]
 [-0.08016399]
 [-0.13644783]
 [-0.11641279]
 [-0.15362583]
 [ 0.        ]]


### Compare with Full Solution

The results from HCB domain decomposition are compared with results from solving the full system. Since no mode truncation has been done, the results from both methods should be equal.

In [18]:
M = np.array([[ 1, 0, 0, 0, 0, 0],
              [ 0, 2, 0, 0, 0, 0],
              [ 0, 0, 2, 0, 0, 0],
              [ 0, 0, 0, 2, 0, 0],
              [ 0, 0, 0, 0, 2, 0],
              [ 0, 0, 0, 0, 0, 1]])

K = np.array([[ 1.2,-1.2, 0.0, 0.0, 0.0, 0.0],
              [-1.2, 2.4,-1.2, 0.0, 0.0, 0.0],
              [ 0.0,-1.2, 2.4,-1.2, 0.0, 0.0],
              [ 0.0, 0.0,-1.2, 2.4,-1.2, 0.0],
              [ 0.0, 0.0, 0.0,-1.2, 2.4, 0.0],
              [ 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]])

F = np.array([[1],
              [1],
              [1],
              [1],
              [1],
              [0]])

u_F = np.linalg.solve( (K - omega * omega * M), F)

print(u_F)

[[-0.32278686]
 [-0.08016399]
 [-0.13644783]
 [-0.11641279]
 [-0.15362583]
 [-0.        ]]
